## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [1]:
import pyconll
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
import nltk

In [2]:
full_train = pyconll.load_from_file(r'D:\train\pos/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file(r'D:\train\pos/ru_syntagrus-ud-dev.conllu')

In [3]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [4]:
len(full_test),len(full_train)

(6584, 48814)

In [5]:
# Приведем данные с списку списков, где слова будут кортежами
fdata_train = []
for sent in full_train:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [6]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [7]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8772537323492737

In [8]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger,verbose=True)
bigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=3165, backoff=89.74%, pruning=98.45%]


0.8829828463586425

In [9]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger,verbose=True)
trigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=4013, backoff=92.13%, pruning=98.76%]


0.882081353418933

In [10]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger,verbose=True)
trigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=5953, backoff=91.77%, pruning=98.16%]


0.881769622215482

In [11]:
bigram_tagger = BigramTagger(fdata_train, backoff=trigram_tagger,verbose=True)
bigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=1183, backoff=90.22%, pruning=99.42%]


0.8827385164964783

In [12]:
unigram_tagger = UnigramTagger(fdata_train,backoff=bigram_tagger,verbose=True)
unigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=561, backoff=83.58%, pruning=99.48%]


0.8773801098641864

In [13]:
unigram_tagger = UnigramTagger(fdata_train,backoff=trigram_tagger,verbose=True)
unigram_tagger.evaluate(fdata_test)

[Trained Unigram tagger: size=561, backoff=83.58%, pruning=99.48%]


0.8773801098641864

In [14]:
# Выделим отдельно списки токенов и с списки меток в трейне и тесте
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [15]:
train_tok[:5],train_label[:5]

(['Анкета', '.', 'Начальник', 'областного', 'управления'],
 ['NOUN', 'PUNCT', 'NOUN', 'ADJ', 'NOUN'])

In [127]:
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np

In [17]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [18]:
train_enc_labels,test_enc_labels,le.classes_

(array([ 7, 13,  7, ...,  9,  7, 13], dtype=int64),
 array([ 7, 13,  1, ..., 10, 16, 13], dtype=int64),
 array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
        'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
        'VERB', 'X'], dtype='<U6'))

In [19]:
countevctorizer = CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 1),dtype = np.float32 )
X_train = countevctorizer.fit_transform(train_tok)
X_test = countevctorizer.transform(test_tok)
X_train.shape,len(train_tok)

((871526, 98088), 871526)

In [130]:
%%time
lr = LogisticRegression(random_state=0,max_iter=2000,n_jobs = 15)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)

accuracy_score(test_enc_labels, pred)

Wall time: 0 ns


In [131]:
%%time
lgbc = LGBMClassifier()
lgbc.fit(X_train, train_enc_labels)
pred = lgbc.predict(X_test)
accuracy_score(test_enc_labels, pred)


Wall time: 0 ns


In [128]:
vectrasers = {'Count_word_lowercase_True_(1,1)': CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 1),dtype = np.float32 ),
              'Count_word_lowercase_True_(1,2)': CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 2),dtype = np.float32 ),
              'Count_word_lowercase_True_(1,3)': CountVectorizer(lowercase=True,analyzer='word',ngram_range=(1, 3),dtype = np.float32 ),
              'Count_word_lowercase_False_(1,1)': CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 1),dtype = np.float32 ),
              'Count_word_lowercase_False_(1,2)': CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 2),dtype = np.float32 ),
              'Count_word_lowercase_False_(1,3)': CountVectorizer(lowercase=False,analyzer='word',ngram_range=(1, 3),dtype = np.float32 ),
              'Count_char_wb_lowercase_False_(1,4)': CountVectorizer(lowercase=False,analyzer='char_wb',ngram_range=(1, 4),dtype = np.float32 ),
              'Count_char_wb_lowercase_False_(1,6)': CountVectorizer(lowercase=False,analyzer='char_wb',ngram_range=(1, 6),dtype = np.float32 ),
              'Count_char_wb_lowercase_False_(2,10)': CountVectorizer(lowercase=False,analyzer='char_wb',ngram_range=(2, 10),dtype = np.float32 ),
              'Hash_char_(1,5)_f_500':HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000),
              'Hash_word_(1,1)_f_500':HashingVectorizer(ngram_range=(1, 1), analyzer='word', n_features=1000),
              'Hash_word_(1,2)_f_500':HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=1000),
              'Hash_word_(1,3)_f_500':HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000),
              'Hash_char_(1,3)_f_500':HashingVectorizer(ngram_range=(1, 10), analyzer='char', n_features=1000),
    
}


In [143]:
import pandas as pd


In [144]:
result = pd.DataFrame()

In [146]:
%%time

for vec in vectrasers:
    X_train = vectrasers[vec].fit_transform(train_tok)
    X_test = vectrasers[vec].transform(test_tok) 
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)

    l = accuracy_score(test_enc_labels, pred)
    result = result.append({'vect':vec,'model':'lr','res':l},ignore_index=True)
    
    lgbc.fit(X_train, train_enc_labels)
    
    pred = lgbc.predict(X_test)
    b = accuracy_score(test_enc_labels, pred)
    
    result = result.append({'vect':vec,'model':'lg','res':b},ignore_index=True)
    
    
    

Wall time: 3h 29min 40s


In [149]:
result.sort_values('res',ascending=False)

,vect,model,res
14,"Count_char_wb_lowercase_False_(1,6)",lr,0.963300
16,"Count_char_wb_lowercase_False_(2,10)",lr,0.963098
12,"Count_char_wb_lowercase_False_(1,4)",lr,0.962407
18,"Hash_char_(1,5)_f_500",lr,0.888122
26,"Hash_char_(1,3)_f_500",lr,0.871702
27,"Hash_char_(1,3)_f_500",lg,0.811259
4,"Count_word_lowercase_True_(1,3)",lr,0.757507
2,"Count_word_lowercase_True_(1,2)",lr,0.757507
0,"Count_word_lowercase_True_(1,1)",lr,0.756749
10,"Count_word_lowercase_False_(1,3)",lr,0.750329


Как можно заметить прекрасные результаты дает логистическая регрессия на коунт векторах по буквенно (1,6),(2,10),(1,4). В пределах слова.  
Бустинг вообще работает только с хэш ветором. В любом случае analyzer='word', хуже чем char


# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [27]:
import corus
from corus import load_ne5
from razdel import tokenize
import pandas as pd

In [21]:
patch_coll_5 = r'D:\train\pos\collection5/'

In [22]:
records = load_ne5(patch_coll_5)
next(records)

Ne5Markup(
    id='001',
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[Ne5Span(
         index='T1',
         type='GEOPOLIT',
         start=0,
         st

процедуры обработки взять из вебинарного ноутбука

In [25]:
words_docs = []
docs_words=[]
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    docs_words.append([rec.text,words]) 
    words_docs.extend(words)

In [31]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_docs = pd.DataFrame(docs_words, columns=['text', 'tag'])

In [32]:
df_words


,word,tag
0,Комиссар,OUT
1,СЕ,ORG
2,критикует,OUT
3,ограничительную,OUT
4,политику,OUT
...,...,...
265342,замглавы,OUT
265343,Бердска,LOC
265344,Владимир,PER
265345,Штоп,PER


In [33]:
df_docs

,text,tag
0,Комиссар СЕ критикует ограничительную политику...,"[[Комиссар, OUT], [СЕ, ORG], [критикует, OUT],..."
1,"Пулеметы, автоматы и снайперские винтовки изъя...","[[Пулеметы, OUT], [,, OUT], [автоматы, OUT], [..."
2,4 октября назначены очередные выборы Верховног...,"[[4, OUT], [октября, OUT], [назначены, OUT], [..."
3,Следственное управление при прокуратуре требуе...,"[[Следственное, ORG], [управление, ORG], [при,..."
4,В Нижегородской области осудили бывшего началь...,"[[В, OUT], [Нижегородской, LOC], [области, LOC..."
...,...,...
994,"Депутат от ""ЕР"": К отставке А.Сердюкова причас...","[[Депутат, OUT], [от, OUT], ["", ORG], [ЕР, ORG..."
995,\r\nСи Цзиньпин избран генсеком Коммунистическ...,"[[Си, PER], [Цзиньпин, PER], [избран, OUT], [г..."
996,"""Ведомости"" узнали о смене лидера московских е...","[["", MEDIA], [Ведомости, MEDIA], ["", MEDIA], [..."
997,СМИ узнали о кутежах туркменского чиновника на...,"[[СМИ, MEDIA], [узнали, OUT], [о, OUT], [кутеж..."


In [34]:
df_words['tag'].value_counts()

OUT         219112
PER          21196
ORG          13650
LOC           4567
GEOPOLIT      4342
MEDIA         2480
Name: tag, dtype: int64

In [81]:
def show_ner_text(n):
    print("TEXT")
    print()
    print(df_docs.text.loc[n])
    print()
    print('NER')  
    print()
    print([i for i in df_docs.tag.loc[n] if i[1]!='OUT'])
    l = {i[1] for i in df_docs.tag.loc[n]}-{"OUT"}
    print()
    print('wich one ner')      
    print(l)
    print(len(l))

In [82]:
show_ner_text(0)

TEXT

Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах

05/08/2008 10:32

МОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.

"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.

По сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".

Комиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, оказались разделенными".



In [83]:
def ner_nltk(n):
    print("TEXT")
    text = df_docs.text.loc[n]
    print()
    print(text)
    print()
    tokens = nltk.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    ner = {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(pos) if hasattr(chunk, 'label') }
    print('NER')  
    print()
    print(ner)
    l = {i[1] for i in df_docs.tag.loc[n]}-{"OUT"}
    print()
#     print('wich one ner')      
#     print(l)
#     print(len(l))
    

In [84]:
ner_nltk(0)

TEXT

Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах

05/08/2008 10:32

МОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.

"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.

По сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".

Комиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, оказались разделенными".



In [118]:
from sklearn import model_selection, preprocessing, linear_model

train_x, test_x, train_y , test_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [119]:
test_y

array([4, 3, 4, ..., 4, 4, 4])

In [120]:
train_y

array([3, 4, 4, ..., 4, 4, 4])

In [121]:
encoder.classes_

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype=object)

In [123]:
train_x,test_x

(193306    стабильности
 166325       предложил
 230145               :
 41673          Якунина
 12465             УМВД
               ...     
 199945     Вернадского
 15519          обороны
 30787                .
 75177            стоит
 172894           Ранее
 Name: word, Length: 199010, dtype: object,
 94725        перед
 235370    институт
 100508           .
 79742            .
 51996            В
             ...   
 260598       когда
 178456      статье
 179377     говорил
 260346      Вместе
 19679        выбыл
 Name: word, Length: 66337, dtype: object)

In [92]:
result_ner = pd.DataFrame()

In [134]:
%%time

for vec in vectrasers:
    X_train = vectrasers[vec].fit_transform(train_x)
    X_test = vectrasers[vec].transform(test_x) 
    lr.fit(X_train, train_y)
    pred = lr.predict(X_test)

    l = accuracy_score(test_y, pred)
    result_ner = result_ner.append({'vect':vec,'model':'lr','res':l},ignore_index=True)
    print(result_ner)
    
    lgbc.fit(X_train, train_y)
    
    pred = lgbc.predict(X_test)
    b = accuracy_score(test_y, pred)
    
    result_ner = result_ner.append({'vect':vec,'model':'lg','res':b},ignore_index=True)
    print(result_ner)

                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
2  Count_word_lowercase_True_(1,1)    lg  0.872801
                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
2  Count_word_lowercase_True_(1,1)    lg  0.872801
3  Count_word_lowercase_True_(1,2)    lr  0.910548
                              vect model       res
0  Count_word_lowercase_True_(1,1)    lr  0.910337
1  Count_word_lowercase_True_(1,1)    lr  0.910337
2  Count_word_lowercase_True_(1,1)    lg  0.872801
3  Count_word_lowercase_True_(1,2)    lr  0.910548
4  Count_word_lowercase_True_(1,2)    lg  0.872801
                              vect model       res
0  Count_word_lowercase_True_(1

                                    vect model       res
0        Count_word_lowercase_True_(1,1)    lr  0.910337
1        Count_word_lowercase_True_(1,1)    lr  0.910337
2        Count_word_lowercase_True_(1,1)    lg  0.872801
3        Count_word_lowercase_True_(1,2)    lr  0.910548
4        Count_word_lowercase_True_(1,2)    lg  0.872801
5        Count_word_lowercase_True_(1,3)    lr  0.910548
6        Count_word_lowercase_True_(1,3)    lg  0.872801
7       Count_word_lowercase_False_(1,1)    lr  0.914166
8       Count_word_lowercase_False_(1,1)    lg  0.873992
9       Count_word_lowercase_False_(1,2)    lr  0.914316
10      Count_word_lowercase_False_(1,2)    lg  0.873992
11      Count_word_lowercase_False_(1,3)    lr  0.914271
12      Count_word_lowercase_False_(1,3)    lg  0.873992
13   Count_char_wb_lowercase_False_(1,4)    lr  0.956193
14   Count_char_wb_lowercase_False_(1,4)    lg  0.948777
15   Count_char_wb_lowercase_False_(1,6)    lr  0.956872
16   Count_char_wb_lowercase_Fa

                                    vect model       res
0        Count_word_lowercase_True_(1,1)    lr  0.910337
1        Count_word_lowercase_True_(1,1)    lr  0.910337
2        Count_word_lowercase_True_(1,1)    lg  0.872801
3        Count_word_lowercase_True_(1,2)    lr  0.910548
4        Count_word_lowercase_True_(1,2)    lg  0.872801
5        Count_word_lowercase_True_(1,3)    lr  0.910548
6        Count_word_lowercase_True_(1,3)    lg  0.872801
7       Count_word_lowercase_False_(1,1)    lr  0.914166
8       Count_word_lowercase_False_(1,1)    lg  0.873992
9       Count_word_lowercase_False_(1,2)    lr  0.914316
10      Count_word_lowercase_False_(1,2)    lg  0.873992
11      Count_word_lowercase_False_(1,3)    lr  0.914271
12      Count_word_lowercase_False_(1,3)    lg  0.873992
13   Count_char_wb_lowercase_False_(1,4)    lr  0.956193
14   Count_char_wb_lowercase_False_(1,4)    lg  0.948777
15   Count_char_wb_lowercase_False_(1,6)    lr  0.956872
16   Count_char_wb_lowercase_Fa

In [135]:
result_ner.sort_values('res',ascending=False)

,vect,model,res
17,"Count_char_wb_lowercase_False_(2,10)",lr,0.956947
15,"Count_char_wb_lowercase_False_(1,6)",lr,0.956872
13,"Count_char_wb_lowercase_False_(1,4)",lr,0.956193
16,"Count_char_wb_lowercase_False_(1,6)",lg,0.949048
14,"Count_char_wb_lowercase_False_(1,4)",lg,0.948777
18,"Count_char_wb_lowercase_False_(2,10)",lg,0.947269
20,"Hash_char_(1,5)_f_500",lg,0.931139
28,"Hash_char_(1,3)_f_500",lg,0.930145
9,"Count_word_lowercase_False_(1,2)",lr,0.914316
11,"Count_word_lowercase_False_(1,3)",lr,0.914271


In [136]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

CountVectorizer(analyzer='char_wb', dtype=<class 'numpy.float32'>,
                lowercase=False, ngram_range=(2, 10))

In [139]:
X_train = vectrasers['Count_char_wb_lowercase_False_(2,10)'].fit_transform(train_x)

In [140]:
X_train

<199010x425805 sparse matrix of type '<class 'numpy.float32'>'
	with 6043691 stored elements in Compressed Sparse Row format>

In [148]:
vectrasers['Count_char_wb_lowercase_False_(2,10)'].get_feature_names()[10000:10050]

[' Вениз',
 ' Венизе',
 ' Венизел',
 ' Венизело',
 ' Венизелос',
 ' Вер',
 ' Вере',
 ' Верет',
 ' Верете',
 ' Веретен',
 ' Веретенн',
 ' Веретенни',
 ' Верещ',
 ' Вереща',
 ' Верещак',
 ' Верещак ',
 ' Верн',
 ' Верна',
 ' Вернад',
 ' Вернадс',
 ' Вернадск',
 ' Вернадско',
 ' Верну',
 ' Вернув',
 ' Вернувш',
 ' Вернувши',
 ' Вернувшис',
 ' Веро',
 ' Верон',
 ' Верони',
 ' Вероник',
 ' Вероника',
 ' Вероника ',
 ' Вероя',
 ' Вероят',
 ' Вероятн',
 ' Вероятне',
 ' Вероятнее',
 ' Вероятно',
 ' Вероятно ',
 ' Вероятны',
 ' Вероятные',
 ' Верх',
 ' Верхн',
 ' Верхне',
 ' Верхнее',
 ' Верхнее ',
 ' Верхню',
 ' Верхнюю',
 ' Верхнюю ']